In [1]:
pip install -U spacy

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install Language

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install decorator

Note: you may need to restart the kernel to use updated packages.


In [5]:
import spacy
from spacy.lang.en import English
from spacy.pipeline import EntityRuler
from spacy.language import Language
import json

In [6]:
def load_data(file):
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)
    return (data)

In [7]:
def save_data(file, data):
    with open(file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

In [37]:
commodities = load_data("data/commodities/commodities.json")
print(commodities)

['aluminium', 'amber', 'brent crude', 'cattle', 'cobalt', 'cocoa', 'coffee', 'copper', 'corn', 'cotton', 'crude oil', 'ethanol', 'gold', 'grain', 'heating oil', 'hogs', 'lead', 'lithium', 'milk', 'molydbenum', 'natural gas', 'nickel', 'oats', 'palladium', 'palm oil', 'platinum', 'propane', 'rapeseed', 'rice', 'rubber', 'silver', 'soybeans', 'sugar', 'tin', 'wheat', 'wool', 'zinc']


In [9]:
def create_training_data(file, type):
    data = load_data(file)
    patterns = []
    for item in data:
        pattern = {
            "label": type,
            "pattern": item
        }
        patterns.append(pattern)
    return (patterns)

In [10]:
@Language.factory("entity_ruler")
def entity_ruler(nlp, name):
    ruler = EntityRuler(nlp)
    return ruler

ValueError: [E004] Can't set up pipeline component: a factory for 'entity_ruler' already exists. Existing factory: <function make_entity_ruler at 0x7fc031177b80>. New factory: <function entity_ruler at 0x7fc0316f7790>

In [11]:
def generate_rules(patterns):
    nlp = English()
    ruler = nlp.add_pipe("entity_ruler")
    ruler.add_patterns(patterns)
    nlp.to_disk("commodities_ner")

In [28]:
patterns = create_training_data("data/commodities/commodities.json", "COMMODITY")
print(patterns)

[{'label': 'COMMODITY', 'pattern': 'aluminium'}, {'label': 'COMMODITY', 'pattern': 'amber'}, {'label': 'COMMODITY', 'pattern': 'brent crude'}, {'label': 'COMMODITY', 'pattern': 'cattle'}, {'label': 'COMMODITY', 'pattern': 'cobalt'}, {'label': 'COMMODITY', 'pattern': 'cocoa'}, {'label': 'COMMODITY', 'pattern': 'coffee'}, {'label': 'COMMODITY', 'pattern': 'copper'}, {'label': 'COMMODITY', 'pattern': 'corn'}, {'label': 'COMMODITY', 'pattern': 'cotton'}, {'label': 'COMMODITY', 'pattern': 'crude oil'}, {'label': 'COMMODITY', 'pattern': 'ethanol'}, {'label': 'COMMODITY', 'pattern': 'gold'}, {'label': 'COMMODITY', 'pattern': 'grain'}, {'label': 'COMMODITY', 'pattern': 'heating oil'}, {'label': 'COMMODITY', 'pattern': 'hogs'}, {'label': 'COMMODITY', 'pattern': 'lead'}, {'label': 'COMMODITY', 'pattern': 'lithium'}, {'label': 'COMMODITY', 'pattern': 'milk'}, {'label': 'COMMODITY', 'pattern': 'molydbenum'}, {'label': 'COMMODITY', 'pattern': 'natural gas'}, {'label': 'COMMODITY', 'pattern': 'nicke

In [29]:
generate_rules(patterns)

In [30]:
nlp = spacy.load("commodities_ner")

In [31]:
def test_model(model, text):
    doc = nlp(text)
    results = []
    for ent in doc.ents:
        results.append(ent.text)
    return results

In [35]:
ie_data = {}

with open("data/commodities/ft-commodities-articles.txt") as f:
    text = f.read()
    articles = text.split("\n")
    for article in articles:
        articleUuid, articleBodyText = article.split("|||")
        segments = articleBodyText.split("||")
        hits = []
        for segment in segments:
            results = test_model(nlp, segment)
            for result in results:
                hits.append(result)
        ie_data[articleUuid] = hits

In [36]:
print(ie_data)

{'74114a6c-8f28-11e7-9084-d0c17942ba93': ['milk'], 'f116e1e8-c58b-11da-b675-0000779e2340': ['gold', 'copper', 'gold'], '1514a6be-a578-11e3-8070-00144feab7de': ['sugar', 'lead', 'lead'], 'e4ab221a-77c6-11e9-bbad-7c18c0ea0201': ['gold', 'gold'], '9db12abc-3b09-11e3-87fa-00144feab7de': ['lead'], 'b022158e-a635-11e3-8a2a-00144feab7de': ['lead'], '71eb184e-da6a-11dc-9bb9-0000779fd2ac': ['lead', 'copper'], 'a3e2f875-5080-408b-ba56-0a7428bab2b1': ['gold', 'gold', 'silver', 'silver', 'coffee'], 'eb8be120-de5f-11e7-a8a4-0a1e63a52f9c': ['silver'], '67c75706-da8d-11dc-9bb9-0000779fd2ac': ['lead', 'cotton', 'cotton', 'wool', 'wool', 'cotton', 'wool', 'wool', 'wool', 'cotton', 'corn', 'wool', 'cotton', 'cotton', 'wool'], 'c97151b4-eb63-11dc-9493-0000779fd2ac': ['lead'], '9e8bdf98-7032-11e6-9ac1-1055824ca907': ['aluminium', 'lead', 'aluminium', 'aluminium', 'aluminium', 'aluminium', 'aluminium', 'aluminium', 'aluminium', 'aluminium', 'aluminium', 'aluminium', 'aluminium', 'aluminium', 'aluminium', '

In [34]:
save_data("data/commodities/commodities_data.json", ie_data)